In [1]:
%run excute_nonZS_MaxMin.ipynb
%matplotlib inline
import csv
import statsmodels.api as sm
import pandas as pd

C:\Users\xzhan\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\xzhan\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
# x_values Normalizaion 
from sklearn.preprocessing import MinMaxScaler
def rescale(dic): # input x is a dictionary - sorted node index: value
    minmax_scale = MinMaxScaler(feature_range=(0, 1))
    x = list(dic.values())
    x = np.array(x)
    x = x.reshape(-1, 1)
    x_scale = minmax_scale.fit_transform(x).flatten()
    dic_scale = dict(zip(dic.keys(),x_scale))
    return dic_scale  # output

# Creat Metrics for Regression

################### create shortest path from all nodes to max selected node v1 ############
def shortest_pth(G, v1):
    nxG = nx.from_numpy_matrix(G)  
    paths = nx.single_source_shortest_path(nxG, v1)
    # the length of shortest path from v2 to v1
    lenths =[]
    for v in paths.items():
        lenth = len(v[1])
        lenths.append(lenth)  
    PathLen = dict(zip(paths.keys(),lenths))
    #print("path_lenth",PathLen)
    return PathLen

def metrics(G, v1):

    nxG = nx.from_numpy_matrix(G) 
    # G = nx.karate_club_graph()
   # print("_______________Degree Centrality_____________________________")  
    deg_centrality = nx.degree_centrality(nxG)
    sortedDict = sorted(deg_centrality.items(), key=lambda x:x[1])
    converted_dict = dict(sortedDict)
    # print(converted_dict)
    #print("                           ")
   # print("_______________Closeness Rank_____________________________")
    close_centrality = nx.closeness_centrality(nxG)
    sortedDict1 = sorted(close_centrality.items(), key=lambda x:x[1])
    converted_dict1 = dict(sortedDict1)
    # print(converted_dict1)
    #print("                           ")
   # print("_______________Page Rank_____________________________")
    pr = nx.eigenvector_centrality(nxG)
    sortedDict2 = sorted(pr.items(), key=lambda x:x[1])
    converted_dict2 = dict(sortedDict2)
    # print(converted_dict2)
    #print("                           ")
   # print("_______________Opinion Extremity_____________________________")
    def gap(op, n):
        ones = np.ones((n, 1))
        x = op - (np.dot(np.transpose(op),ones)/n) * ones
        return abs(x)
    gap = gap(s,n)
    my_gap = {index: value for index, value in enumerate(gap)}
    sorting_gap = sorted(my_gap.items(), key=lambda x:x[1], reverse=True)
    sorted_gap = dict(sorting_gap)
    # print("opinion - mean")
    # print(sorted_gap)
    
   # print("________________Shortest Path_____________________________")
    PathLen = shortest_pth(G, v1)
    
    
    # creat a dict node:shortest length to v1(max selected node)
    scaled_MinPath = rescale(PathLen) # rescale the shortest path to the range (0,1) for regression
    ##print("scaled path lenths",scaled_MinPath)
    # sorting all varibles based on the node index
    scaled_MinPaths_sort = dict(sorted(scaled_MinPath.items(), key=lambda x:x[0]))
    #print("scaled_paths_sort",scaled_MinPaths_sort)
    
    converted_dict = dict(sorted(converted_dict.items(), key=lambda x:x[0]))
    converted_dict1 = dict(sorted(converted_dict1.items(), key=lambda x:x[0]))
    converted_dict2 = dict(sorted(converted_dict2.items(), key=lambda x:x[0]))
    sorted_gap = dict(sorted(sorted_gap.items(), key=lambda x:x[0]))
    
    return (converted_dict, converted_dict1, converted_dict2, sorted_gap, scaled_MinPaths_sort)

    


In [3]:
# Find the ground truth of Min's action - knowing Max chooses v1, what Min's action will be?
def actual_rank(s, n, v1, max_opinion):

    L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
    A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
    op = copy.copy(s)
    op[v1] = max_opinion
    all_1 = list(range(n))    # for all agent 
    C1 = [x for x in all_1 if x != v1]  # for the vertice that Maximizer has not touched
    innat_por = obj_polarization(A, s, n) # Calculate innate polarization
    results = []
    for v2 in C1:   
        (changed_opinion, por) = derivate_s(op,n,v2,A)   # find the best new_op option  
        por_1 = por - innat_por     # append the change of the polarization
        results.append((v2, por_1))
    results.append((v1,0)) # minimizer cannot choose v1, but we need all nodes for the dataframe
    actual_Y = dict(results) # now we have - node:polarization
    
    # most of v are in sequential order, but v1 might not be in order, so we need to sort it
    sorted_actual_Y = sorted(actual_Y.items(), key=lambda x:x[0])
    actual_Y_dict = dict(sorted_actual_Y)  # dictionary -  node:polarization
   # print("actual_Y_dict",actual_Y_dict) 

    # create the rank of the Y values based on minimizer's choice 
    actual_Y_order = dict(sorted(actual_Y.items(), key=lambda x:x[1]))  #sort - list nodes in sequential of the polariz.

    node_ranks= dict(zip(actual_Y_order.keys(), all_1)) # dictionary -  node:polarization rank(replace polarz. with rank)
    actual_ranks = dict(sorted(node_ranks.items(), key=lambda x:x[0])) # sort - list node in sequential node index
    
    return (actual_Y_dict, actual_ranks)  # return two dictionary, 1. sequential node: polarization, 
                                                                # 2. seq. node: pol rank

In [4]:
df = pd.DataFrame({'keys': list(converted_dict2.keys())})
print(df)
for i in range(2):
    s = make_innat_opinions(n)
    G = make_random_network(n)
    (v1, v2, max_opinion, min_opinion, max_pol) = MaxMin_play(s,n,G)
    (converted_dict, converted_dict1, converted_dict2, sorted_gap, scaled_MinPaths_sort) = metrics(G, v1) 
   # print('where')
    ##################################the actual rank based on polarization ####################
    (actual_Y_dict, actual_ranks) = actual_rank(s, n, v1, max_opinion)
     # rescale the polarization change to the range (0,1) for regression
    rescaled_dict = rescale(actual_Y_dict)
    #max_pred = sorted(np.unique(a+b+c))
    
    df['x11'] = df['keys'].apply(lambda x: sorted_gap[x][0])
    df['x22'] = df['keys'].apply(lambda x: converted_dict2[x])
    df['x33'] = df['keys'].apply(lambda x: scaled_MinPaths_sort[x])
    df['y_value'] = df['keys'].apply(lambda x: actual_Y_dict[x])

    print(df)
    

NameError: name 'converted_dict2' is not defined

In [5]:
ue': [actual_Y_dict[x] for x in converted_dict2.keys()]})

for i in range(1, 2):  # Change the range if you want more iterations
    s = make_innat_opinions(n)
    G = make_random_network(n)
    (v1, v2, max_opinion, min_opinion, max_pol) = MaxMin_play(s, n, G)
    (converted_dict, converted_dict1, converted_dict2, sorted_gap, scaled_MinPaths_sort) = metrics(G, v1)
    
    # Calculate actual ranks and rescale polarization change
    (actual_Y_dict, actual_ranks) = actual_rank(s, n, v1, max_opinion)
    rescaled_dict = rescale(actual_Y_dict)
    
    # Create a temporary DataFrame for the current iteration's data
    temp_df = pd.DataFrame({'keys': list(converted_dict2.keys()),
                            'x11': [sorted_gap[x][0] for x in converted_dict2.keys()],
                            'x22': [converted_dict2[x] for x in converted_dict2.keys()],
                            'x33': [scaled_MinPaths_sort[x] for x in converted_dict2.keys()],
                            'y_value': [actual_Y_dict[x] for x in converted_dict2.keys()]})
    
    # Concatenate the temporary DataFrame to the main DataFrame
    df = pd.concat([df, temp_df], ignore_index=True)

# Print the final DataFrame
print(df)


                                
Maximizer finds its target agent:
Max Action:    Agent29 's opinion 0.5411009471018632 changed to 1
Min Action:    Agent18 's opinion 0.05618440720807116 changed to 0.7072943666421232
Network reaches equilibrium Polarization: 0.022919381937473733
                                
Maximizer finds its target agent:
Max Action:    Agent29 's opinion 0.27004348197857564 changed to 0
Min Action:    Agent28 's opinion 0.033540726648598085 changed to 1
Network reaches equilibrium Polarization: 0.08503668742763593
    keys       x11       x22       x33   y_value
0      0  0.044516  0.193785  1.000000  0.017544
1      1  0.313042  0.162840  1.000000  0.017779
2      2  0.062595  0.205713  0.666667  0.017621
3      3  0.330022  0.213409  1.000000  0.016599
4      4  0.110308  0.250376  0.666667  0.017289
5      5  0.212714  0.228595  0.666667  0.017633
6      6  0.142566  0.259323  0.666667  0.017468
7      7  0.053996  0.222373  0.666667  0.017433
8      8  0.348